In [12]:
import xlwings as xw

def copy_wespac_values(old_file, new_file, output_file, version='3.2'):
    # Define the sheets, columns, and last rows to copy from the old to the new file
    sheets_to_copy = {
        'OF': {'column': 'D', 'last_row': 174},
        'F': {'column': 'D', 'last_row': 345},
        'S': {'column': 'F', 'last_row': 101}
    }

    # Open the old and new WESP-AC files using xlwings
    app = xw.App(visible=False)
    old_wespac = app.books.open(old_file)
    new_wespac = app.books.open(new_file)

    try:
        for sheet, info in sheets_to_copy.items():
            column = info['column']
            last_row = info['last_row']
            
            old_sheet = old_wespac.sheets[sheet]
            new_sheet = new_wespac.sheets[sheet]

            # Unprotect the sheet if it's protected
            try:
                new_sheet.api.Unprotect()
            except Exception as e:
                print(f"Failed to unprotect the sheet '{sheet}': {e}")

            # Handle version-specific adjustments
            if version == '3.2' and sheet == 'OF':
                # OF: Shift range for OF38 from 170-173 to 171-174
                for row in range(1, 170):
                    old_value = old_sheet.range(f'{column}{row}').value
                    new_sheet.range(f'{column}{row}').value = old_value

                # Shift the range for OF38 by one row
                for row in range(170, 174):
                    old_value = old_sheet.range(f'{column}{row}').value
                    new_sheet.range(f'{column}{row+1}').value = old_value  # Shift to new range 171-174

            elif version == '3.2' and sheet == 'F':
                # F: Skip row 345 (F68) as it's new in 3.4
                for row in range(1, 345):
                    if row == 345:
                        continue  # Skip F68 (345)
                    old_value = old_sheet.range(f'{column}{row}').value
                    new_sheet.range(f'{column}{row}').value = old_value

            else:
                # For other versions or sheets where no specific modification is needed
                for row in range(1, last_row + 1):
                    old_value = old_sheet.range(f'{column}{row}').value
                    if old_value is not None:
                        new_sheet.range(f'{column}{row}').value = old_value

            # Protect the sheet again after modifying
            new_sheet.api.Protect()

        # Save the new WESP-AC file with the copied values
        new_wespac.save(output_file)
        print(f"Values successfully copied from {old_file} to {output_file} (Version: {version})")

    finally:
        # Close both workbooks and quit the app
        old_wespac.close()
        new_wespac.close()
        app.quit()

# Define file paths
old_wespac_file = 'WESP-AC3.2.xlsx'
new_wespac_file = 'wespac_3.4.xlsx'
output_wespac_file = 'wespac_3.4_filled_values.xlsx'

# Run the function to copy values, specifying version 3.2
copy_wespac_values(old_wespac_file, new_wespac_file, output_wespac_file, version='3.2')


Values successfully copied from WESP-AC3.2.xlsx to wespac_3.4_filled_values.xlsx (Version: 3.2)
